In [7]:
import pandas as pd

In [8]:
data = pd.read_csv('data/MinMax_Scaler_result.csv',encoding='cp949')

In [9]:

from sklearn.model_selection import train_test_split

In [10]:
features = data[[ 'CBSCORE', 'CREDITOTAMT', 'YSALEAMT',
       'ESTMM', 'IMSAAMT', 'IMJUAMT',  'BUJUAMT',
       'BU1TOTAMT', 'GAMT', 'KOSPI', '환율', 'GDP', 
       '실업률', '물가지수', '국고채', '금리', '유가등락률',
       '소비자심리지수','ONEHOT']] #GKIND 갖고 가나요?.. 아뇨..ㅋㅋㅋ
act = data['ACTCD'] 

In [11]:

train_x, test_x,train_y, test_y = train_test_split(features, act,
stratify=act,train_size=0.7,test_size=0.3,random_state=1)

print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(27381, 19) (11736, 19) (27381,) (11736,)


In [12]:
'''
1. ENN
2. ADASYN
3. BORDERLINE SMOTE
4. Tomeklinks
5 SOTEENN
6.SMOTETOMEK
'''
from imblearn.over_sampling import BorderlineSMOTE,ADASYN
from imblearn.combine import SMOTEENN,SMOTETomek
from imblearn.under_sampling import EditedNearestNeighbours, TomekLinks
data_sets={}
enn = EditedNearestNeighbours()
ada = ADASYN(random_state=42)
smte = SMOTEENN(random_state=42)
smtt = SMOTETomek(random_state=42)
tk = TomekLinks()
sm = BorderlineSMOTE(random_state=42)
enn_under_train_x,enn_under_train_y = enn.fit_resample(train_x,train_y)
ada_over_train_x, ada_over_train_y = ada.fit_resample(train_x,train_y)
smte_combine_train_x,smte_combine_train_y = smte.fit_resample(train_x,train_y)
smtt_combine_train_x,smtt_combine_train_y = smtt.fit_resample(train_x,train_y)
tk_under_train_x,tk_under_train_y = tk.fit_resample(train_x,train_y)
sm_over_train_x, sm_over_train_y = sm.fit_resample(train_x,train_y)
data_sets['enn']=[enn_under_train_x,enn_under_train_y]
data_sets['ada']=[ada_over_train_x,ada_over_train_y]
data_sets['smte']=[smte_combine_train_x,smte_combine_train_y]
data_sets['smtt']=[smtt_combine_train_x,smtt_combine_train_y]
data_sets['tk']=[tk_under_train_x,tk_under_train_y]
data_sets['sm']=[sm_over_train_x,sm_over_train_y]
'''
over_train_x : 학습용 독립변수
over_train_y : 학습용 종속변수
'''
keys_list= data_sets.keys()
accuracies = []
matrix_kd_tree_uniform_BOST = []

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import precision_score,recall_score,f1_score,roc_auc_score,confusion_matrix, classification_report


In [14]:
for i in keys_list:
    model = RandomForestClassifier(n_estimators=200,max_depth=50,max_features=1)
    model.fit(data_sets[i][0], data_sets[i][1])
                            
                    # 결정계수
    relation_square = model.score(data_sets[i][0], data_sets[i][1])
    print('결정계수 : ', relation_square)
                    
                    # 예측
    pred_y = model.predict(test_x)
    pr = precision_score(y_true=test_y,y_pred=pred_y)
    r = recall_score(y_true=test_y,y_pred=pred_y)
    f1 = f1_score(y_true=test_y,y_pred=pred_y)
    rca = roc_auc_score(y_true=test_y,y_score=pred_y)
    cfm = confusion_matrix(y_true=test_y,y_pred=pred_y)
    matrix_kd_tree_uniform_BOST.append((i,pr,r,f1,rca,model.score(test_x, test_y),cfm))
                # 정확도 확인
    print(i)
    print('정확도 : ', metrics.accuracy_score(test_y, pred_y))
    print(classification_report(test_y, pred_y, target_names=['사고보증', '정상보증']))
    print('\n')

결정계수 :  0.9991934761179317
enn
정확도 :  0.924079754601227
              precision    recall  f1-score   support

        사고보증       0.93      0.99      0.96     10666
        정상보증       0.77      0.24      0.36      1070

    accuracy                           0.92     11736
   macro avg       0.85      0.62      0.66     11736
weighted avg       0.91      0.92      0.91     11736



결정계수 :  0.9989321190385042
ada
정확도 :  0.9023517382413088
              precision    recall  f1-score   support

        사고보증       0.93      0.96      0.95     10666
        정상보증       0.45      0.32      0.37      1070

    accuracy                           0.90     11736
   macro avg       0.69      0.64      0.66     11736
weighted avg       0.89      0.90      0.89     11736



결정계수 :  1.0
smte
정확도 :  0.8479890933878664
              precision    recall  f1-score   support

        사고보증       0.94      0.89      0.91     10666
        정상보증       0.29      0.48      0.36      1070

    accuracy          

In [15]:
matrix_kd_tree_uniform_BOST

[('enn',
  0.7720364741641338,
  0.23738317757009345,
  0.3631165117941386,
  0.6151757440447505,
  0.924079754601227,
  array([[10591,    75],
         [  816,   254]], dtype=int64)),
 ('ada',
  0.45013123359580054,
  0.3205607476635514,
  0.3744541484716157,
  0.6406385212159872,
  0.9023517382413088,
  array([[10247,   419],
         [  727,   343]], dtype=int64)),
 ('smte',
  0.29435483870967744,
  0.47757009345794393,
  0.36421952957947257,
  0.6813595826374662,
  0.8479890933878664,
  array([[9441, 1225],
         [ 559,  511]], dtype=int64)),
 ('smtt',
  0.48011363636363635,
  0.31588785046728973,
  0.3810597519729425,
  0.6407866029009991,
  0.906441717791411,
  array([[10300,   366],
         [  732,   338]], dtype=int64)),
 ('tk',
  0.9011857707509882,
  0.2130841121495327,
  0.34467120181405897,
  0.6053701078280008,
  0.9261247443762781,
  array([[10641,    25],
         [  842,   228]], dtype=int64)),
 ('sm',
  0.4769687964338782,
  0.3,
  0.36833046471600683,
  0.63349896